In [13]:
import os
project_name = "reco-chef"; branch = "30music"; account = "sparsh-ai"
project_path = os.path.join('/content', project_name)

In [15]:
!git checkout "{branch}"

Branch '30music' set up to track remote branch '30music' from 'origin'.
Switched to a new branch '30music'


In [1]:
import os
project_name = "reco-tut-sess"; branch = "main"; account = "sparsh-ai"
project_path = os.path.join('/content', project_name)

In [14]:
if not os.path.exists(project_path):
    !pip install -U -q dvc dvc[gdrive]
    !cp -r /content/drive/MyDrive/git_credentials/. ~
    path = "/content/" + project_name; 
    !mkdir "{path}"
    %cd "{path}"
    !git init
    !git remote add origin https://github.com/"{account}"/"{project_name}".git
    !git pull origin "{branch}"
    !git checkout main
else:
    %cd "{project_path}"

/content/reco-chef
Initialized empty Git repository in /content/reco-chef/.git/
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 114 (delta 32), reused 108 (delta 27), pack-reused 0
Receiving objects: 100% (114/114), 22.83 KiB | 11.41 MiB/s, done.
Resolving deltas: 100% (32/32), done.
From https://github.com/sparsh-ai/reco-chef
 * branch            30music    -> FETCH_HEAD
 * [new branch]      30music    -> origin/30music
error: pathspec 'main' did not match any file(s) known to git.


In [25]:
!git status

On branch 30music
Your branch is up to date with 'origin/30music'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	data/bronze/30music/

nothing added to commit but untracked files present (use "git add" to track)


In [26]:
!git add . && git commit -m 'commit' && git push origin "{branch}"

usic 998d26c] commit
 3 files changed, 24 insertions(+)
 create mode 100644 data/bronze/30music/.gitignore
 create mode 100644 data/bronze/30music/sessions.parquet.snappy.dvc
 create mode 100644 data/bronze/30music/sessions_sample_10.parquet.snappy.dvc
Counting objects: 8, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 976 bytes | 976.00 KiB/s, done.
Total 8 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 1 local object.
To https://github.com/sparsh-ai/reco-chef.git
   491304b..998d26c  30music -> 30music


In [ ]:
!dvc pull

In [12]:
!dvc commit && dvc push

outputs ['ml1m/v0/movies.dat'] of stage: 'ml1m/v0/movies.dat.dvc' changed. Are you sure you want to commit it? [y/n] ERROR: interrupted by the user


---

In [ ]:
import numpy as np
import pandas as pd
import datetime
import calendar
import time
from collections import Counter

In [ ]:
class SessionDataset:
    def __init__(self, df, seed=42):
        self.data = df.copy()
        self._standardize()
        self.seed = seed
        self.train = None
        self.test = None

    def _standardize(self):
        col_names = ['session_id', 'user_id', 'item_id', 'ts'] + self.data.columns.values.tolist()[4:]
        self.data.columns = col_names

    def _add_months(self, sourcedate, months):
        month = sourcedate.month - 1 + months
        year = int(sourcedate.year + month / 12)
        month = month % 12 + 1
        day = min(sourcedate.day, calendar.monthrange(year, month)[1])
        return datetime.date(year, month, day)

    def filter_by_time(self, last_months=0):
        if last_months > 0:
            lastdate = datetime.datetime.fromtimestamp(self.data.ts.max())
            firstdate = self._add_months(lastdate, -last_months)
            initial_unix = time.mktime(firstdate.timetuple())
            self.data = self.data[self.data['ts'] >= initial_unix]

    def convert_to_sequence(self, topk=0):
        c = Counter(list(self.data['item_id']))
        if topk > 1:
            keeper = set([x[0] for x in c.most_common(topk)])
            self.data = self.data[self.data['item_id'].isin(keeper)]

        # group by session id and concat song_id
        groups = self.data.groupby('session_id')

        # convert item ids to string, then aggregate them to lists
        aggregated = groups['item_id'].agg(sequence = lambda x: list(map(str, x)))
        init_ts = groups['ts'].min()
        users = groups['user_id'].min()  # it's just fast, min doesn't actually make sense

        self.data = aggregated.join(init_ts).join(users)
        self.data.reset_index(inplace=True)

    def get_stats(self):
        cnt = Counter()
        _stats = []
        self.data.sequence.map(cnt.update);
        sequence_length = self.data.sequence.map(len).values
        n_sessions_per_user = self.data.groupby('user_id').size()

        _stats.append('Number of items: {}'.format(len(cnt)))
        _stats.append('Number of users: {}'.format(self.data.user_id.nunique()))
        _stats.append('Number of sessions: {}'.format(len(self.data)) )

        _stats.append('Session length:\n\tAverage: {:.2f}\n\tMedian: {}\n\tMin: {}\n\tMax: {}'.format(
            sequence_length.mean(), 
            np.quantile(sequence_length, 0.5), 
            sequence_length.min(), 
            sequence_length.max()))

        _stats.append('Sessions per user:\n\tAverage: {:.2f}\n\tMedian: {}\n\tMin: {}\n\tMax: {}'.format(
            n_sessions_per_user.mean(), 
            np.quantile(n_sessions_per_user, 0.5), 
            n_sessions_per_user.min(), 
            n_sessions_per_user.max()))

        _stats.append('Most popular items: {}'.format(cnt.most_common(5)))
        _stats =  '\n'.join(_stats)
        
        return _stats

    def random_holdout(self, split=0.8):
        """
        Split sequence data randomly
        :param split: the training percentange
        """
        self.data = self.data.sample(frac=1, random_state=self.seed)
        nseqs = len(self.data)
        train_size = int(nseqs * split)
        self.train = self.data[:train_size]
        self.test = self.data[train_size:]

    def temporal_holdout(self, ts_threshold):
        """
        Split sequence data using timestamps
        :param ts_threshold: the timestamp from which test sequences will start
        """
        self.train = self.data.loc[self.data['ts'] < ts_threshold]
        self.test = self.data.loc[self.data['ts'] >= ts_threshold]
        self.train, self.test = self._clean_split(self.train, self.test)

    def last_session_out_split(self,
                               user_key='user_id', 
                               session_key='session_id',
                               time_key='ts'):
        """
        Assign the last session of every user to the test set and the remaining ones to the training set
        """
        sessions = self.data.sort_values(by=[user_key, time_key]).groupby(user_key)[session_key]
        last_session = sessions.last()
        self.train = self.data[~self.data.session_id.isin(last_session.values)].copy()
        self.test = self.data[self.data.session_id.isin(last_session.values)].copy()
        self.train, self.test = self._clean_split(self.train, self.test)

    def _clean_split(self, train, test):
        """
        Remove new items from the test set.
        :param train: The training set.
        :param test: The test set.
        :return: The cleaned training and test sets.
        """
        train = train.copy()
        test = test.copy()
        train_items = set()
        train['sequence'].apply(lambda seq: train_items.update(set(seq)))
        test['sequence'] = test['sequence'].apply(lambda seq: [it for it in seq if it in train_items])
        return train, test

In [ ]:
df = pd.read_parquet('./data/bronze/30music/sessions_sample_10.parquet.snappy')

In [ ]:
df.head()

,session_id,user_id,song_id,ts,playtime
0,1902204,4,16,1421163674,274
1,1902204,4,17,1421163948,250
2,1902204,4,18,1421164198,271
3,780919,10,60,1411009500,228
4,780919,10,61,1411014936,206


In [ ]:
df.info()

### Scratch testing

In [ ]:
test_df = [[1,1,1,'2015-01-13',10],
             [2,1,1,'2015-02-13',20],
             [2,1,3,'2015-02-13',5],
             [3,1,3,'2015-02-14',15],
             [4,2,1,'2014-12-13',10],
             [5,2,2,'2015-02-10',2],
             [5,2,1,'2015-02-10',9],
             [5,2,3,'2015-02-10',3],
             [5,2,3,'2015-02-10',7],
             ]
test_df = pd.DataFrame(test_df)
test_df.columns = ['session_id', 'user_id', 'item_id', 'ts', 'playtime']
test_df.ts = test_df.ts.apply(dt_int)
test_df

,session_id,user_id,item_id,ts,playtime
0,1,1,1,1421107200,10
1,2,1,1,1423785600,20
2,2,1,3,1423785600,5
3,3,1,3,1423872000,15
4,4,2,1,1418428800,10
5,5,2,2,1423526400,2
6,5,2,1,1423526400,9
7,5,2,3,1423526400,3
8,5,2,3,1423526400,7


In [ ]:
xx = SessionDataset(test_df)
# xx.filter_by_time(last_months=0)
# xx.filter_by_time(last_months=1)
xx.convert_to_sequence()
# xx.convert_to_sequence(topk=2)
# xx.get_stats()
# xx.data#.to_dict()

# xx.random_holdout(0.6)
xx.temporal_holdout(1423600000)
# xx.temporal_holdout(1423500000)
# xx.last_session_out_split()
# display(xx.train)
# display(xx.test)
xx.train.to_dict('list')
# xx.train

### Unit testing

In [ ]:
import unittest
import pandas as pd
from pandas.testing import assert_frame_equal


test_df = [[1,1,1,'2015-01-13',10],
             [2,1,1,'2015-02-13',20],
             [2,1,3,'2015-02-13',5],
             [3,1,3,'2015-02-14',15],
             [4,2,1,'2014-12-13',10],
             [5,2,2,'2015-02-10',2],
             [5,2,1,'2015-02-10',9],
             [5,2,3,'2015-02-10',3],
             [5,2,3,'2015-02-10',7],
             ]
test_df = pd.DataFrame(test_df)
test_df.columns = ['session_id', 'user_id', 'item_id', 'ts', 'playtime']


def _dt_int(dt, tm='00:00:00'):
    """converts date (& time) to integer"""
    return int(datetime.datetime.strptime('{} {}'.format(dt,tm), '%Y-%m-%d %H:%M:%S').strftime("%s"))

test_df.ts = test_df.ts.apply(dt_int)


class TestDataset(unittest.TestCase):
    def setUp(self):
        pass

    def testFilterByTimeNoFilter(self):
        """If month=0, do not remove any rows
        passing first n rows of the test_df,
        expected not to remove any rows
        """
        _dataset = SessionDataset(test_df.iloc[:9,:])
        _dataset.filter_by_time(last_months=0)
        assert_frame_equal(test_df.iloc[:,:], _dataset.data)

    def testFilterByTimeFilter(self):
        """If month>0, remove rows
        passing first n rows of the test_df,
        expected to remove some rows
        """
        _dataset = SessionDataset(test_df.iloc[:9,:])
        _dataset.filter_by_time(last_months=1)
        assert_frame_equal(test_df.iloc[[1,2,3,5,6,7,8],:], _dataset.data)

    def testItemConversionToSequence(self):
        """convert items to a list in time-based sequence
        passing first n rows of the test_df,
        expected as per dictionary frame defined below
        """
        _dataset = SessionDataset(test_df.iloc[:9,:])
        _dataset.convert_to_sequence()
        _expecteddf = pd.DataFrame.from_dict({
            'session_id': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
            'sequence': {0: ['1'], 1: ['1', '3'], 2: ['3'], 3: ['1'], 4: ['2', '1', '3', '3']},
            'ts': {0: 1421107200, 1: 1423785600, 2: 1423872000, 3: 1418428800, 4: 1423526400},
            'user_id': {0: 1, 1: 1, 2: 1, 3: 2, 4: 2}})
        assert_frame_equal(_expecteddf, _dataset.data)        

    def testItemConversionToSequenceTopK(self):
        """convert items to a list in time-based sequence
        filters topk most interacted items
        passing first n rows of the test_df with topk=2,
        expected as per dictionary frame defined below
        """
        _dataset = SessionDataset(test_df.iloc[:9,:])
        _dataset.convert_to_sequence(topk=2)
        _expecteddf = pd.DataFrame.from_dict({
            'session_id': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
            'sequence': {0: ['1'], 1: ['1', '3'], 2: ['3'], 3: ['1'], 4: ['1', '3', '3']},
            'ts': {0: 1421107200, 1: 1423785600, 2: 1423872000, 3: 1418428800, 4: 1423526400},
            'user_id': {0: 1, 1: 1, 2: 1, 3: 2, 4: 2}})
        assert_frame_equal(_expecteddf, _dataset.data)   

    def testDataStatistics(self):
        """generate statistics of the dataset
        passing first n rows of the test_df,
        expected as per string defined below
        expected:
        Number of items: 3\nNumber of users: 2\nNumber of sessions: 
        5\nSession length:\n\tAverage: 1.80\n\tMedian: 1.0\n\tMin: 
        1\n\tMax: 4\nSessions per user:\n\tAverage: 2.50\n\tMedian: 
        2.5\n\tMin: 2\n\tMax: 3\nMost popular items: 
        [('1', 4), ('3', 4), ('2', 1)]"""
        _dataset = SessionDataset(test_df.iloc[:9,:])
        _dataset.convert_to_sequence()
        _generated = _dataset.get_stats()        
        self.assertIn("Number of items: 3", _generated)    
        self.assertIn("Most popular items: [('1', 4), ('3', 4), ('2', 1)]", _generated)    
        self.assertIn("Session length:\n\tAverage: 1.80\n\tMedian: 1.0", _generated)    
        self.assertNotIn("Number of items: 4", _generated)     

    def testRandomSplit(self):
        _dataset = SessionDataset(test_df.iloc[:9,:])
        _dataset.convert_to_sequence()
        _dataset.random_holdout(0.6)
        _expecteddf = pd.DataFrame.from_dict(
            {'session_id': {1: 2, 2: 3, 4: 5},
            'sequence': {1: ['1', '3'], 2: ['3'], 4: ['2', '1', '3', '3']},
            'ts': {1: 1423785600, 2: 1423872000, 4: 1423526400},
            'user_id': {1: 1, 2: 1, 4: 2}}
            )
        _expecteddf = _expecteddf.reindex([1,4,2])
        assert_frame_equal(_expecteddf, _dataset.train)

    def testTemporalSplitThreshold1(self):
        _dataset = SessionDataset(test_df.iloc[:9,:])
        _dataset.convert_to_sequence()
        _dataset.temporal_holdout(1423600000)
        _expecteddf = pd.DataFrame.from_dict(
            {'session_id': {0: 1, 3: 4, 4: 5},
            'sequence': {0: ['1'], 3: ['1'], 4: ['2', '1', '3', '3']},
            'ts': {0: 1421107200, 3: 1418428800, 4: 1423526400},
            'user_id': {0: 1, 3: 2, 4: 2}}
            )
        assert_frame_equal(_expecteddf, _dataset.train) 

    def testTemporalSplitThreshold2(self):
        _dataset = SessionDataset(test_df.iloc[:9,:])
        _dataset.convert_to_sequence()
        _dataset.temporal_holdout(1423500000)
        _expecteddf = pd.DataFrame.from_dict(
            {'session_id': {0: 1, 3: 4},
            'sequence': {0: ['1'], 3: ['1']},
            'ts': {0: 1421107200, 3: 1418428800},
            'user_id': {0: 1, 3: 2}}
        )
        assert_frame_equal(_expecteddf, _dataset.train) 

    def testSessionOutSplit(self):
        _dataset = SessionDataset(test_df.iloc[:9,:])
        _dataset.convert_to_sequence()
        _dataset.last_session_out_split()
        _expecteddf = pd.DataFrame.from_dict(
            {'session_id': {0: 1, 1: 2, 3: 4},
            'sequence': {0: ['1'], 1: ['1', '3'], 3: ['1']},
            'ts': {0: 1421107200, 1: 1423785600, 3: 1418428800},
            'user_id': {0: 1, 1: 1, 3: 2}}
        )
        assert_frame_equal(_expecteddf, _dataset.train)

unittest.main(argv=[''], verbosity=2, exit=False)

testDataStatistics (__main__.TestMoney)
generate statistics of the dataset ... ok
testFilterByTimeFilter (__main__.TestMoney)
If month>0, remove rows ... ok
testFilterByTimeNoFilter (__main__.TestMoney)
If month=0, do not remove any rows ... ok
testItemConversionToSequence (__main__.TestMoney)
convert items to a list in time-based sequence ... ok
testItemConversionToSequenceTopK (__main__.TestMoney)
convert items to a list in time-based sequence ... ok
testRandomSplit (__main__.TestMoney) ... ok
testSessionOutSplit (__main__.TestMoney) ... ok
testTemporalSplitThreshold1 (__main__.TestMoney) ... ok
testTemporalSplitThreshold2 (__main__.TestMoney) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.111s

OK


### Library testing

In [ ]:
import sys
sys.path.insert(0,'./src')

In [ ]:
from src.data import SessionDataset as SD

In [ ]:
df = pd.read_parquet('./data/bronze/30music/sessions_sample_10.parquet.snappy')

In [ ]:
sd = SD(df)

In [ ]:
sd.convert_to_sequence()
stats = sd.get_stats()
print(stats)

Number of items: 9999
Number of users: 14262
Number of sessions: 37877
Session length:
	Average: 9.06
	Median: 7.0
	Min: 3
	Max: 199
Sessions per user:
	Average: 2.66
	Median: 2.0
	Min: 1
	Max: 28
Most popular items: [('2388', 559), ('67', 508), ('531', 489), ('275', 416), ('443', 409)]


### Script testing

In [ ]:
!make setup

python3 setup.py install
running install
running bdist_egg
running egg_info
creating src/src.egg-info
writing src/src.egg-info/PKG-INFO
writing dependency_links to src/src.egg-info/dependency_links.txt
writing top-level names to src/src.egg-info/top_level.txt
writing manifest file 'src/src.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'src/src.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/utils
copying src/utils/__init__.py -> build/lib/utils
copying src/utils/gdrive.py -> build/lib/utils
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/utils
copying build/lib/utils/__init__.py -> build/bdist.linux-x86_64/egg/utils
copying build/lib/utils/gdrive.py -> build/bdist.linux-x86_64/egg/utils
byte-compiling build/bdist.linux-x86_64/egg/utils/__init__.py to __init__.cpython-37.pyc
byte-

In [ ]:
!make test

PYTHONPATH=. pytest
============================= test session starts ==============================
platform linux2 -- Python 2.7.17, pytest-3.6.4, py-1.8.0, pluggy-0.7.1
rootdir: /content/reco-tut-sess, inifile:
collected 10 items                                                             

tests/test_dataset.py F..FFFFFF                                          [ 90%]
tests/test_dummy.py .                                                    [100%]

=================================== FAILURES ===================================
_________________________ TestMoney.testDataStatistics _________________________

self = <test_dataset.TestMoney testMethod=testDataStatistics>

    def testDataStatistics(self):
        """generate statistics of the dataset
            passing first n rows of the test_df,
            expected as per string defined below
            expected:
            Number of items: 3\nNumber of users: 2\nNumber of sessions:
            5\nSession length:\n\tAverage: 1.8